<a href="https://colab.research.google.com/github/kkmax1015/Kaggle/blob/main/Titanic/1_SampleNoteBook_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#目標：ロジスティック回帰を使用してとりあえず予測結果を出してみる
まずはロジスティク回帰を使用して、予測結果の算出までの流れを体験してみる。

###1.データを読み込み、どんな項目があるか見てみる

In [21]:
# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/Colab Notebooks/Titanic/'
!ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Titanic
1_SampleNoteBook_LogisticRegression  data


In [23]:
# PandasをImportする
import pandas as pd

In [28]:
# CSVデータをPandasデータフレームへ読み込む
pd_train = pd.read_csv("./data/train.csv");
pd_test = pd.read_csv("./data/test.csv");

In [27]:
# トレーニング用データの先頭5行を表示
pd_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [29]:
# テスト用データの先頭5行を表示
pd_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [31]:
# それぞれのデータの件数を算出する
print(len(pd_train), len(pd_test))

891 418


トレーニングデータとテストデータの見比べると、テストデータには「Survived」の項目がないことがわかる。これは、「Survived」は予測するべき答え(0：死亡 1：生存)である為。
トレーニング用データには回答が用意されている為、このトレーニングデータを使用して機械学習モデルを作成していく。

データの各項目の意味はKaggleに説明がある為、そちらを参照。

https://www.kaggle.com/c/titanic/data

###2.ロジスティック回帰モデルを作成してみる

scikit-learnのロジスティック回帰 (logistic regression) モデルを使用して、機械学習モデルを作成してみる。

scikit-learn：Pythonのオープンソース機械学習ライブラリ。主要なライブラリは実装されているので、深層学習(ニューラルネットワーク)以外は大体このライブラリで何とかなる。

ロジスティック回帰：回帰という名称がついているが、分類問題で使用される。次の記事を参照。
https://qiita.com/katsu1110/items/e4ef613559f02f183af5

scikit-learnのロジスティック回帰のページは以下。使い方でわからないものがあればまずは下記のページを参照すること。

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

In [32]:
# sklearnのロジスティック回帰をImport
from sklearn.linear_model import LogisticRegression

In [33]:
# ロジスティック回帰モデルの作成(初期化)
# random_state：初期化パラメータ。指定することで学習結果を固定することが出来る。
clf = LogisticRegression(random_state=0)